# Prepare Data for Visualization

Jenna Jordan

13 February 2020

### Purpose

This notebook takes the time-series dataset produced in the "Query_GNI_via_Solr" notebook and transforms the data into the format required for the visualization tool.

Additionally, some extra transformations as required for the statistical analyses are performed.

In [1]:
import pandas as pd
import numpy as np

In [2]:
def reformat_df_for_viz(df, publisher=True):
    if publisher:
        grouping_columns = ['publisher', 'publication_date', 'query']
    else:
        grouping_columns = ['publication_date', 'query']
    
    # get raw article counts
    df_count = df.stack().to_frame()
    df_count.columns = ['count']
    df_count.index.names = grouping_columns
    df_count = df_count.reset_index(grouping_columns)
    
    # get article proportions
    df_prop = df.copy()
    for c in df_prop.columns:
        if c == 'total':
            pass
        else:
            df_prop[c] = df_prop[c] / df_prop.total
    df_prop = df_prop.drop(columns=['total'])

    df_prop = df_prop.stack().to_frame()
    df_prop.columns = ['proportion']
    df_prop.index.names = grouping_columns
    df_prop = df_prop.reset_index(grouping_columns)
    
    # merge counts + proportions
    df_viz = df_count.merge(df_prop, on=grouping_columns, how='outer')
    
    nan_mask = (df_viz['proportion'].isna()) & (df_viz['query']!="total")
    df_viz['count'] = np.where(nan_mask, np.NaN, df_viz['count'])
    
    return df_viz

## BLN Time-Series data

In [12]:
df_bln_total = pd.read_csv('../Data/bln_daily_total.csv', parse_dates=['publication_date'], dtype={'publisher':'category'})

In [13]:
df_bln_queries = pd.read_csv('../Data/query_results_bln-ts_26Feb.csv', parse_dates=['publication_date'], dtype={'publisher':'category'})

### Monthly time-series for the 6 publishers

- 3 publishers are excluded due to irregularities/not enough data
- monthly version is used for the interactive visualization tool

In [14]:
df_bln = df_bln_total.merge(df_bln_queries, on=['publication_date', 'publisher'])

In [15]:
df_bln_monthly = df_bln.set_index('publication_date').groupby('publisher').resample('M').sum()
df_bln_monthly = df_bln_monthly.rename(columns={'BLN_total':'total'})

In [16]:
df_bln_monthly

total  insect_population  insect_decline  \
publisher publication_date                                               
AFP       1991-05-31         4819.0                0.0             0.0   
          1991-06-30         6451.0                0.0             0.0   
          1991-07-31         6595.0                0.0             0.0   
          1991-08-31         6540.0                0.0             0.0   
          1991-09-30         6599.0                0.0             0.0   
...                             ...                ...             ...   
XGNS      2019-04-30        11088.0                2.0             0.0   
          2019-05-31        11592.0                5.0             1.0   
          2019-06-30        10128.0                6.0             2.0   
          2019-07-31        11152.0                6.0             2.0   
          2019-08-31         5535.0                1.0             0.0   

                            pollinator_population  pollinator_decline  \
publisher publication_date                                              
AFP       1991-05-31                          0.0                 0.0   
          1991-06-30                          0.0                 0.0   
          1991-07-31                          0.0                 0.0   
          1991-08-31                          0.0                 0.0   
          1991-09-30                          0.0                 0.0   
...                                           ...                 ...   
XGNS      2019-04-30                          0.0                 0.0   
          2019-05-31                          2.0                 1.0   
          2019-06-30                          2.0                 0.0   
          2019-07-31                          1.0                 0.0   
          2019-08-31                          0.0                 0.0   

                            insect_apocalypse  colony_collapse  \
publisher publication_date                                       
AFP       1991-05-31                      0.0              0.0   
          1991-06-30                      0.0              0.0   
          1991-07-31                      0.0              0.0   
          1991-08-31                      0.0              0.0   
          1991-09-30                      0.0              0.0   
...                                       ...              ...   
XGNS      2019-04-30                      0.0              0.0   
          2019-05-31                      0.0              0.0   
          2019-06-30                      0.0              0.0   
          2019-07-31                      0.0              0.0   
          2019-08-31                      0.0              0.0   

                            climate_change  climate_change_IPCCreport  \
publisher publication_date                                              
AFP       1991-05-31                   2.0                        0.0   
          1991-06-30                   8.0                        0.0   
          1991-07-31                   5.0                        0.0   
          1991-08-31                   5.0                        0.0   
          1991-09-30                   7.0                        0.0   
...                                    ...                        ...   
XGNS      2019-04-30                 156.0                        0.0   
          2019-05-31                 195.0                        1.0   
          2019-06-30                 154.0                        1.0   
          2019-07-31                 174.0                        0.0   
          2019-08-31                  75.0                        5.0   

                            insect_population_studies  
publisher publication_date                             
AFP       1991-05-31                              0.0  
          1991-06-30                              0.0  
          1991-07-31                              0.0  
          1991-08-31        

In [17]:
bln_m_6pubs = reformat_df_for_viz(df_bln_monthly)

In [18]:
bad_publisher_mask = bln_m_6pubs[bln_m_6pubs['publisher'].isin(['IPS', 'UPI', 'SWB'])].index
bln_m_6pubs = bln_m_6pubs.drop(bad_publisher_mask)

In [20]:
bln_m_6pubs

,publisher,publication_date,query,count,proportion
0,AFP,1991-05-31,total,4819.0,NaN
1,AFP,1991-05-31,insect_population,0.0,0.000000
2,AFP,1991-05-31,insect_decline,0.0,0.000000
3,AFP,1991-05-31,pollinator_population,0.0,0.000000
4,AFP,1991-05-31,pollinator_decline,0.0,0.000000
...,...,...,...,...,...
37195,XGNS,2019-08-31,insect_apocalypse,0.0,0.000000
37196,XGNS,2019-08-31,colony_collapse,0.0,0.000000
37197,XGNS,2019-08-31,climate_change,75.0,0.013550
37198,XGNS,2019-08-31,climate_change_IPCCreport,5.0,0.000903


In [9]:
bln_6pubs.to_csv("../Data/Visualize/bln-queries_6pubs_26Feb.csv", index=False)

### for the 6 publishers, but weekly

- Weekly version is used for the final visualizations (used for the publication) produced in R (with ggplot2)

In [31]:
df_bln_weekly = df_bln.set_index('publication_date').groupby('publisher').resample('W').sum()
df_bln_weekly = df_bln_weekly.rename(columns={'BLN_total':'total'})

In [32]:
bln_w_6pubs = reformat_df_for_viz(df_bln_weekly)
bln_w_6pubs = bln_w_6pubs.rename(columns={'BLN_total':'total'})

In [33]:
bad_publisher_mask = bln_w_6pubs[bln_w_6pubs['publisher'].isin(['IPS', 'UPI', 'SWB'])].index
bln_w_6pubs = bln_w_6pubs.drop(bad_publisher_mask)

In [34]:
bln_w_6pubs

,publisher,publication_date,query,count,proportion
0,AFP,1991-05-05,total,152.0,NaN
1,AFP,1991-05-05,insect_population,0.0,0.000000
2,AFP,1991-05-05,insect_decline,0.0,0.000000
3,AFP,1991-05-05,pollinator_population,0.0,0.000000
4,AFP,1991-05-05,pollinator_decline,0.0,0.000000
...,...,...,...,...,...
161575,XGNS,2019-08-18,insect_apocalypse,0.0,0.000000
161576,XGNS,2019-08-18,colony_collapse,0.0,0.000000
161577,XGNS,2019-08-18,climate_change,31.0,0.015446
161578,XGNS,2019-08-18,climate_change_IPCCreport,0.0,0.000000


## For the statistical analysis

- Scott requested the data in this format to better enable analysis in Stata
- Further transforms the weekly time-series dataset

In [35]:
df_count = bln_w_6pubs.drop(columns=['proportion']).set_index(['publisher', 'publication_date', 'query'])
df_count = df_count.unstack(level='query')
df_count.columns = df_count.columns.droplevel()
df_count = df_count.drop(columns=['colony_collapse', 'insect_apocalypse'])
df_count = df_count.add_suffix('_count')

In [36]:
df_prop = bln_w_6pubs.drop(columns=['count']).set_index(['publisher', 'publication_date', 'query'])
df_prop = df_prop.unstack(level='query')
df_prop.columns = df_prop.columns.droplevel()
df_prop = df_prop.drop(columns=['colony_collapse', 'insect_apocalypse', 'total'])
df_prop = df_prop.add_suffix('_proportion')

In [37]:
df_analyze = pd.concat([df_count, df_prop], axis=1)

In [38]:
df_analyze

query                       climate_change_count  \
publisher publication_date                         
AFP       1991-05-05                         0.0   
          1991-05-12                         0.0   
          1991-05-19                         1.0   
          1991-05-26                         0.0   
          1991-06-02                         1.0   
...                                          ...   
XGNS      2019-07-21                        41.0   
          2019-07-28                        37.0   
          2019-08-04                        38.0   
          2019-08-11                        26.0   
          2019-08-18                        31.0   

query                       climate_change_IPCCreport_count  \
publisher publication_date                                    
AFP       1991-05-05                                    0.0   
          1991-05-12                                    0.0   
          1991-05-19                                    0.0   
          1991-05-26                                    0.0   
          1991-06-02                                    0.0   
...                                                     ...   
XGNS      2019-07-21                                    0.0   
          2019-07-28                                    0.0   
          2019-08-04                                    0.0   
          2019-08-11                                    5.0   
          2019-08-18                                    0.0   

query                       insect_decline_count  insect_population_count  \
publisher publication_date                                                  
AFP       1991-05-05                         0.0                      0.0   
          1991-05-12                         0.0                      0.0   
          1991-05-19                         0.0                      0.0   
          1991-05-26                         0.0                      0.0   
          1991-06-02                         0.0                      0.0   
...                                          ...                      ...   
XGNS      2019-07-21                         0.0                      0.0   
          2019-07-28                         2.0                      2.0   
          2019-08-04                         0.0                      1.0   
          2019-08-11                         0.0                      1.0   
          2019-08-18                         0.0                      0.0   

query                       insect_population_studies_count  \
publisher publication_date                                    
AFP       1991-05-05                                    0.0   
          1991-05-12                                    0.0   
          1991-05-19                                    0.0   
          1991-05-26                                    0.0   
          1991-06-02                                    0.0   
...                                                     ...   
XGNS      2019-07-21                                    0.0   
          2019-07-28                                    0.0   
          2019-08-04                                    0.0   
          2019-08-11                                    0.0   
          2019-08-18                                    0.0   

query                       pollinator_decline_count  \
publisher publication_date                             
AFP       1991-05-05                             0.0   
          1991-05-12                             0.0   
          1991-05-19                             0.0   
          1991-05-26                             0.0   
          1991-06-02                             0.0   
...                                              ...   
XGNS      2019-07-21                             0.0   
          2019-07-28                             0.0   
          2019-08-04                             0.0   
          2019-08-11                             0.0   
         

In [39]:
df_analyze.to_csv("../Data/Analyze/BLNqueries_timeseries_weekly_3Mar.csv")